In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import unidecode
import time

from openpyxl import load_workbook
#import basedosdados as bd

pd.options.display.float_format = '{:.2f}'.format
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

Algumas informações:

2014 (com coligação) 2018 (com coligação) - Dep Estaduais - Dep Federais 

2016 (com coligação) 2020 (sem coligação) - Vereadores 

Partidos que tiveram os nomes modificados entre as eleições 2014 e 2018: DC, PODEMOS, AVANTE, MDB AVANTE (PTB), PATRI ** NOVO, REDE e PMB tiveram seus registros aprovados após as eleições 2014 (Talvez a base esteja alimentada pelo numero dos partidos, mas é bom observar isso ao comparar eleições. Além dos novos partidos)

Coligações podem candidatar 200% das vagas disponíveis, não-coligados 150%.


* Considerar apenas candidatos aptos?

# Deputados estaduais

Fonte: https://www.tse.jus.br/eleicoes/estatisticas/repositorio-de-dados-eleitorais-1

In [4]:
path = 'consulta_cand_2014/consulta_cand_2014_BRASIL.csv'
df = pd.read_csv(path, sep=';', encoding='latin1', low_memory=False, decimal=',')

df.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,08/04/2021,11:38:59,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000003276,13819,JAQUELINE MARQUES TEIXEIRA,JAQUELINE MARQUES,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,13,PT,PARTIDO DOS TRABALHADORES,190000000010,PARTIDO ISOLADO,PT,-4,NÃO DIVULGÁVEL,Não divulgável,-3,NÃO DIVULGÁVEL,NaN,nan,-4,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,5,SUPLENTE,Não divulgável,Não divulgável,1047852014,37404120146190000,2,DEFERIDO,2,DEFERIDO,SIM
1,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,AC,AC,ACRE,7,DEPUTADO ESTADUAL,10000000131,65565,LUZIELE ALVES DIAS,LUZIELE,#NULO#,19682140200,PCDOBACRI@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,10000000024,FRENTE POPULAR DO ACRE VI,PC do B / PTB,1,BRASILEIRA NATA,AC,-3,RIO BRANCO,06/05/1970,44.00,1743942453,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),3,PARDA,266,PROFESSOR DE ENSINO MÉDIO,600000,5,SUPLENTE,N,S,52882014,1981620146010000,2,DEFERIDO,2,DEFERIDO,SIM
2,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RS,RS,RIO GRANDE DO SUL,6,DEPUTADO FEDERAL,210000000667,4567,LEUDO IRAJA SANTOS COSTA,LEUDO COSTA,#NULO#,20529457091,LEUDOCOSTA@MEIOAMBIENTELEGAL.COM.BR,12,APTO,2,DEFERIDO,COLIGAÇÃO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,210000000020,UNIDOS PELA ESPERANÇA,PP / PRB / SD / PSDB,1,BRASILEIRA NATA,RS,-3,SANTIAGO,19/10/1955,59.00,37851190450,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRANCA,131,ADVOGADO,4500000,5,SUPLENTE,N,S,315182014,5147120146210000,2,DEFERIDO,2,DEFERIDO,SIM
3,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,GO,GO,GOIÁS,7,DEPUTADO ESTADUAL,90000000500,50111,ARNALDO LUIZ DA SILVA,ARNALDINHO,#NULO#,49062948120,SOLANOCASTRO50@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,90000000021,PARTIDO ISOLADO,PSOL,1,BRASILEIRA NATA,PE,-3,SAO BEBEDITO DO SUL,31/07/1964,50.00,39479221066,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,200000,4,NÃO ELEITO,N,S,303612014,7627020146090000,2,DEFERIDO,2,DEFERIDO,SIM
4,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000000746,70038,JOSÉ JAILSON CRUZ,CAMARÃO,#NULO#,39653501704,JAILSONJCRUZ@IG.COM.BR,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,70,PT do B,PARTIDO TRABALHISTA DO BRASIL,190000000015,PARTIDO ISOLADO,PT do B,1,BRASILEIRA NATA,AL,-3,PENEDO,28/08/1952,62.00,49091390353,2,MASCULINO,7,SUPERIOR INCOMPLETO,3,CASADO(A),1,BRANCA,999,OUTROS,1000000,5,SUPLENTE,N,N,759262014,27392120146190000,2,DEFERIDO,2,DEFERIDO,SIM


Abaixo as colunas do dataframe com a consulta de candidatos:

In [39]:
df.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
       'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO', 'NM_EMAIL',
       'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'CD_NACIONALIDADE',
       'DS_NACIONALIDADE', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO',
       'NM_MUNICIPIO_NASCIMENTO', 'DT_NASCIMENTO', 'NR_IDADE_DATA_POSSE',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
       'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO',
       '

### 2014

Abaixo estão as coligações presentes em cada estado e a quantidade de candidaturas por gênero em cada coligação, além da quantidade de candidaturas por gênero representadas por partidos isolados:

In [26]:
df_dep_estadual_coligacoes_genero_2014  = df[
    df['DS_CARGO'] == 'DEPUTADO ESTADUAL'
].groupby(['SG_UF','TP_AGREMIACAO','NM_COLIGACAO'])['DS_GENERO'].value_counts()

df_dep_estadual_coligacoes_genero_2014.head()

SG_UF  TP_AGREMIACAO  NM_COLIGACAO                    DS_GENERO
AC     COLIGAÇÃO      ALIANÇA POR UM ACRE MELHOR II   MASCULINO    50
                                                      FEMININO     21
                      Aliança Por Um Acre Melhor III  MASCULINO    40
                                                      FEMININO     18
                      FRENTE POPULAR DO ACRE 5        MASCULINO    24
Name: DS_GENERO, dtype: int64

Abaixo o número de candidaturas por gênero vinculadas a partidos isolados com a quantidade por cada partido

In [31]:
df_dep_estadual_partido_isolado_genero_2014 = df[
    (df['DS_CARGO'] == 'DEPUTADO ESTADUAL') &
    (df['NM_COLIGACAO'] == 'PARTIDO ISOLADO')
].groupby(['SG_UF', 'SG_PARTIDO'])['DS_GENERO'].value_counts()

df_dep_estadual_partido_isolado_genero_2014.head()

SG_UF  SG_PARTIDO  DS_GENERO
AC     PDT         MASCULINO    25
                   FEMININO     11
       PP          MASCULINO    28
                   FEMININO     13
       PRP         MASCULINO    26
Name: DS_GENERO, dtype: int64

Vagas para deputado estadual por estado: 

“O número de Deputados à Assembleia Legislativa corresponderá ao triplo da representação do Estado na Câmara dos Deputados e, atingido o número de trinta e seis, será acrescido de tantos quantos forem os Deputados Federais acima de doze”, diz o artigo 27 da Constituição federal.

In [7]:
df['SG_UF'].unique()

array(['RJ', 'AC', 'RS', 'GO', 'MG', 'PI', 'SP', 'PR', 'AL', 'AM', 'PE',
       'ES', 'MA', 'MS', 'TO', 'PA', 'AP', 'DF', 'SC', 'RR', 'RO', 'PB',
       'BA', 'MT', 'CE', 'SE', 'RN', 'BR'], dtype=object)

In [8]:
df_depestadual = df[df['DS_CARGO'] == 'DEPUTADO ESTADUAL']
df_depestadual.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,08/04/2021,11:38:59,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000003276,13819,JAQUELINE MARQUES TEIXEIRA,JAQUELINE MARQUES,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,13,PT,PARTIDO DOS TRABALHADORES,190000000010,PARTIDO ISOLADO,PT,-4,NÃO DIVULGÁVEL,Não divulgável,-3,NÃO DIVULGÁVEL,NaN,nan,-4,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,5,SUPLENTE,Não divulgável,Não divulgável,1047852014,37404120146190000,2,DEFERIDO,2,DEFERIDO,SIM
1,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,AC,AC,ACRE,7,DEPUTADO ESTADUAL,10000000131,65565,LUZIELE ALVES DIAS,LUZIELE,#NULO#,19682140200,PCDOBACRI@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,10000000024,FRENTE POPULAR DO ACRE VI,PC do B / PTB,1,BRASILEIRA NATA,AC,-3,RIO BRANCO,06/05/1970,44.00,1743942453,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),3,PARDA,266,PROFESSOR DE ENSINO MÉDIO,600000,5,SUPLENTE,N,S,52882014,1981620146010000,2,DEFERIDO,2,DEFERIDO,SIM
3,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,GO,GO,GOIÁS,7,DEPUTADO ESTADUAL,90000000500,50111,ARNALDO LUIZ DA SILVA,ARNALDINHO,#NULO#,49062948120,SOLANOCASTRO50@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,90000000021,PARTIDO ISOLADO,PSOL,1,BRASILEIRA NATA,PE,-3,SAO BEBEDITO DO SUL,31/07/1964,50.00,39479221066,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,200000,4,NÃO ELEITO,N,S,303612014,7627020146090000,2,DEFERIDO,2,DEFERIDO,SIM
4,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000000746,70038,JOSÉ JAILSON CRUZ,CAMARÃO,#NULO#,39653501704,JAILSONJCRUZ@IG.COM.BR,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,70,PT do B,PARTIDO TRABALHISTA DO BRASIL,190000000015,PARTIDO ISOLADO,PT do B,1,BRASILEIRA NATA,AL,-3,PENEDO,28/08/1952,62.00,49091390353,2,MASCULINO,7,SUPERIOR INCOMPLETO,3,CASADO(A),1,BRANCA,999,OUTROS,1000000,5,SUPLENTE,N,N,759262014,27392120146190000,2,DEFERIDO,2,DEFERIDO,SIM
5,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,MG,MG,MINAS GERAIS,7,DEPUTADO ESTADUAL,130000000169,33100,MARCOS NICÁCIO GOMES,MARCOS DO IORGUTE,#NULO#,3866027605,#NULO#,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,33,PMN,PARTIDO DA MOBILIZAÇÃO NACIONAL,130000000004,PARTIDO ISOLADO,PMN,1,BRASILEIRA NATA,MG,-3,SÃO SEBASTIÃO DO SACRAMENTO,09/01/1978,37.00,126247940230,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,9,DIVORCIADO(A),4,AMARELA,402,"VENDEDOR PRACISTA, REPRESENTANTE, CAIXEIRO-VIAJANTE E ASSEMELHADOS",5000000,5,SUPLENTE,N,S,1482332014,4091920146130000,2,DEFERIDO,2,DEFERIDO,SIM


In [9]:
df['TP_AGREMIACAO'].unique()

array(['PARTIDO ISOLADO', 'COLIGAÇÃO'], dtype=object)

In [10]:
df_depestadual[
    (df_depestadual['SG_UF'] == 'RJ') & 
    (df_depestadual['TP_AGREMIACAO'] == 'COLIGAÇÃO')
].groupby('NM_COLIGACAO')['DS_COMPOSICAO_COLIGACAO'].value_counts()

NM_COLIGACAO                          DS_COMPOSICAO_COLIGACAO
EM DEFESA DA FAMÍLIA                  PSC / PTC                  138
JUNTOS PELO TRABALHISMO               PTB / PTN                  145
PACTO PELO RIO DE JANEIRO E O BRASIL  PSDB / PPS / DEM           145
PARA MELHORAR TEM QUE MUDAR           PRTB / PPL                 121
PR E PROS                             PR / PROS                  137
UNIDOS PELA FAMILIA                   PSDC / PMN                 100
Name: DS_COMPOSICAO_COLIGACAO, dtype: int64

In [11]:
df_depestadual[
    (df_depestadual['SG_UF'] == 'RJ') & 
    (df_depestadual['TP_AGREMIACAO'] == 'PARTIDO ISOLADO')
].groupby('NM_COLIGACAO')['DS_COMPOSICAO_COLIGACAO'].value_counts()

NM_COLIGACAO     DS_COMPOSICAO_COLIGACAO
PARTIDO ISOLADO  PHS                        105
                 PSL                        104
                 PRB                         96
                 PSB                         96
                 PT do B                     95
                 PC do B                     86
                 PDT                         75
                 PT                          74
                 PATRIOTA                    73
                 PSOL                        71
                 PV                          67
                 PMDB                        65
                 PP                          65
                 SD                          64
                 PRP                         62
                 PSD                         38
                 PSTU                        19
                 PCB                          2
Name: DS_COMPOSICAO_COLIGACAO, dtype: int64

In [12]:
for cidade in df_depestadual['SG_UF'].unique():
    df_depestadual[df_depestadual['SG_UF'] == cidade]

Coligações

In [12]:
df_coligacoes['tipo_agremiacao'].unique()

array(['coligacao', 'partido isolado'], dtype=object)

In [28]:
df_coligacoes[df_coligacoes['tipo_agremiacao'] == 'coligacao'].head()

,turno,tipo_eleicao,id_municipio_tse,cargo,tipo_agremiacao,numero_partido,sigla_partido,nome_partido,coligacao,sequencial_coligacao,ano,sigla_uf
0,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,30000000009,O Amapá Que Queremos,2014,AP
1,eleicao ordinaria,1,None,deputado federal,coligacao,10,PRB,Partido Republicano Brasileiro,30000000014,Avança Amapá,2014,AP
2,eleicao ordinaria,1,None,deputado federal,coligacao,45,PSDB,Partido Da Social Democracia Brasileira,30000000005,Juntos Pelo Desenvolvimento Pela Paz E Pela Vida,2014,AP
3,eleicao ordinaria,1,None,deputado federal,coligacao,33,PMN,Partido Da Mobilização Nacional,30000000009,O Amapá Que Queremos,2014,AP
4,eleicao ordinaria,1,None,deputado federal,coligacao,50,PSOL,Partido Socialismo E Liberdade,30000000019,Frente Popular A Favor Do Amapa,2014,AP


In [32]:
df_coligacoes[df_coligacoes['numero_partido'] == '54']

,turno,tipo_eleicao,id_municipio_tse,cargo,tipo_agremiacao,numero_partido,sigla_partido,nome_partido,coligacao,sequencial_coligacao,ano,sigla_uf
0,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,30000000009,O Amapá Que Queremos,2014,AP
41,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,20000000005,Caminhando Com O Povo,2014,AL
90,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,70000000031,Respeito Por Brasília 1,2014,DF
124,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,40000000022,Renovação E Experiência,2014,AM
135,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,130000000019,Minas Quer Mudança,2014,MG
171,eleicao ordinaria,1,None,deputado federal,partido isolado,54,PPL,Partido Pátria Livre,100000000024,None,2014,MA
203,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,60000000017,Ptc - Pen - Pv - Pt Do B - Prtb - Pmn - Ppl,2014,CE
228,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,80000000012,Frente Por Um Espirito Santo Melhor,2014,ES
262,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,50000000005,Um Novo Caminho Para A Bahia,2014,BA
288,eleicao ordinaria,1,None,deputado federal,coligacao,54,PPL,Partido Pátria Livre,120000000033,Mato Grosso Do Sul Com A Força De Todos Ii,2014,MS


In [33]:
df_dep_est[
    (df_dep_est['sigla_uf'] == 'AP') &
    (df_dep_est['numero_partido'] == '54')
]

,tipo_eleicao,id_candidato_bd,cpf,titulo_eleitoral,sequencial_candidato,numero_candidato,nome_candidato,nome_urna_candidato,numero_partido,sigla_partido,cargo,id_municipio_tse,situacao,ocupacao,data_nasc,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nasc,municipio_nasc,email,raca,ano,sigla_uf
5923,eleicao ordinaria,105890,01457965828,001238222518,30000000187,54122,Paulo Edson Cardoso Bezerra,Paulo Edson,54,PPL,deputado estadual,None,deferido,mecanico de manutencao,1958-03-24,57,masculino,ensino medio completo,casado(a),brasileira,AP,Macapa,pplsantanaap@gmailcom,parda,2014,AP
5924,eleicao ordinaria,786106,20880294272,000628352526,30000000211,54444,Maria Silva Do Carmo,Maria Do Carmo,54,PPL,deputado estadual,None,deferido,outros,1965-05-26,49,feminino,ensino medio completo,solteiro(a),brasileira,PA,Afua,None,parda,2014,AP
5925,eleicao ordinaria,1100355,38958350300,002949432518,30000000186,54123,Haroldo Wilson Leal Abdon,Haroldo Abdon Da Topfiat,54,PPL,deputado estadual,None,deferido,outros,1970-05-23,44,masculino,ensino medio completo,solteiro(a),brasileira,PA,Chaves,haroldoabdontopfiat@hotmailcom,parda,2014,AP
5926,eleicao ordinaria,1162932,43279589215,001717252518,30000000555,54380,Edineuza De Souza Brito Da Silva,Edineuza Brito,54,PPL,deputado estadual,None,deferido,outros,1974-04-05,40,feminino,ensino medio completo,casado(a),brasileira,PA,Chaves,None,parda,2014,AP
5927,eleicao ordinaria,1363516,59256958287,002573902534,30000000188,54000,Edson Santos Ferreira,Pastor Edson,54,PPL,deputado estadual,None,deferido,outros,1978-09-30,36,masculino,ensino medio completo,casado(a),brasileira,AP,Macapa,esantosferreira2014@bolcombr,parda,2014,AP
5928,eleicao ordinaria,1407287,62685066268,000342812550,30000000210,54222,Aldete Moreira Macedo,Aldete,54,PPL,deputado estadual,None,deferido,outros,1964-12-17,50,feminino,ensino medio completo,casado(a),brasileira,PA,Chaves,None,parda,2014,AP
5929,eleicao ordinaria,1671634,86889311272,004493372500,30000000209,54456,Delma De Souza Brito,Delma,54,PPL,deputado estadual,None,deferido,outros,1985-01-03,30,feminino,ensino medio completo,solteiro(a),brasileira,AP,Macapá,None,branca,2014,AP


In [20]:
df_coligacoes.groupby('sequencial_coligacao')['sigla_partido'].value_counts()

sequencial_coligacao                                                      sigla_partido
 Salve Roraima I                                                          DEM              1
                                                                          PP               1
                                                                          PTB              1
+Minas                                                                    PEN              1
                                                                          PHS              1
                                                                          PRP              1
                                                                          PT do B          1
A Experiência Faz A Mudança 1                                             PMDB             1
                                                                          PSD              1
                                                                          P

In [26]:
df_dep_est.head(1)

,tipo_eleicao,id_candidato_bd,cpf,titulo_eleitoral,sequencial_candidato,numero_candidato,nome_candidato,nome_urna_candidato,numero_partido,sigla_partido,cargo,id_municipio_tse,situacao,ocupacao,data_nasc,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nasc,municipio_nasc,email,raca,ano,sigla_uf
0,eleicao ordinaria,9377,00126677654,098230520221,130000001770,21456,Adailton Moreira,Adailton,21,PCB,deputado estadual,None,deferido,professor de ensino fundamental,1972-03-22,43,masculino,ensino superior completo,solteiro(a),brasileira,MG,Rio Espera,moreiraadailton@bolcombr,branca,2014,MG


In [27]:
df_dep_est['sigla_partido'].value_counts()

PT         871
PSB        849
PMDB       822
PSC        742
PV         741
PC do B    738
PSDB       688
PSOL       671
PDT        669
PRP        663
PHS        657
PEN        651
PTB        633
PSL        585
PP         563
PTC        557
PR         554
PT do B    538
PSDC       534
PTN        504
PPS        462
PRB        449
PSD        447
DEM        420
PRTB       415
SD         378
PMN        362
PROS       339
PPL        331
PSTU       130
PCB         39
PCO         10
Name: sigla_partido, dtype: int64

In [29]:
df_dep_est[
    df_dep_est['sigla_partido'] == 'DEM'
]

,tipo_eleicao,id_candidato_bd,cpf,titulo_eleitoral,sequencial_candidato,numero_candidato,nome_candidato,nome_urna_candidato,numero_partido,sigla_partido,cargo,id_municipio_tse,situacao,ocupacao,data_nasc,idade,genero,instrucao,estado_civil,nacionalidade,sigla_uf_nasc,municipio_nasc,email,raca,ano,sigla_uf
3156,eleicao ordinaria,20090,00272035572,000289790558,50000000715,25118,Luiz Barbosa De Deus,Luiz De Deus,25,DEM,deputado estadual,None,deferido,deputado,1938-06-05,76,masculino,ensino superior completo,casado(a),brasileira,SE,Brejo Grande,None,parda,2014,BA
3157,eleicao ordinaria,142055,01996770500,038729510523,50000000711,25155,Heraldo Eduardo Rocha,Heraldo Rocha,25,DEM,deputado estadual,None,deferido,medico,1942-02-25,73,masculino,ensino superior completo,viuvo(a),brasileira,RJ,Itaperuna,None,branca,2014,BA
3158,eleicao ordinaria,161101,02278214594,117647290523,50000000774,25024,Alecsandro De Souza Santos,Léo Kret Do Brasil,25,DEM,deputado estadual,None,deferido,empresario,1983-12-09,31,masculino,ensino fundamental completo,solteiro(a),brasileira,BA,Salvador,None,parda,2014,BA
3159,eleicao ordinaria,544640,09582169591,025963670582,50000000723,25456,Targino Machado Pedreira Filho,Targino Machado,25,DEM,deputado estadual,None,deferido,medico,1952-09-04,62,masculino,ensino superior completo,divorciado(a),brasileira,BA,Salvador,None,branca,2014,BA
3160,eleicao ordinaria,602176,11427280568,010546190574,50000000756,25019,Jose Nilton Azevedo Leal,Capitao Azevedo,25,DEM,deputado estadual,None,indeferido,servidor publico estadual,1952-08-08,62,masculino,ensino superior completo,solteiro(a),brasileira,BA,Ilheus,None,parda,2014,BA
3161,eleicao ordinaria,610473,11756721572,038630790566,50000000720,25125,Sandra Maria Rocha Borges Conceição,Profa. Sandra Borges,25,DEM,deputado estadual,None,deferido,professor de ensino medio,1956-02-10,59,feminino,ensino superior completo,divorciado(a),brasileira,BA,Salvador,None,parda,2014,BA
3162,eleicao ordinaria,622105,12225240582,064718180523,50000000709,25333,Edson Fonseca Diniz,Edy Diniz,25,DEM,deputado estadual,None,deferido,professor de ensino superior,1958-09-27,56,masculino,ensino superior completo,casado(a),brasileira,BA,Salvador,None,parda,2014,BA
3163,eleicao ordinaria,673937,14462028520,034915950582,50000000819,25199,Ana Virginia Tavares Lisbôa De Cerqueira,Ana Virginia Cerqueira,25,DEM,deputado estadual,None,deferido,outros,1959-08-03,55,feminino,ensino superior incompleto,divorciado(a),brasileira,BA,Salvador,None,branca,2014,BA
3164,eleicao ordinaria,687975,15169219504,018287980507,50000000713,25025,José Luciano Santos Ribeiro,Luciano Ribeiro,25,DEM,deputado estadual,None,deferido,advogado,1960-12-22,54,masculino,ensino superior completo,casado(a),brasileira,BA,Caculé,None,branca,2014,BA
3165,eleicao ordinaria,792928,21374996572,013221680507,50000000714,25966,Luciano Antonio Pinheiro,Luciano Da Locar,25,DEM,deputado estadual,None,deferido,comerciante,1964-06-11,50,masculino,ensino medio completo,casado(a),brasileira,BA,Amargosa,None,parda,2014,BA


## 2018 (com e sem coligação)

# Deputados Federais

Número de vagas em cada estado: https://g1.globo.com/politica/eleicoes/2018/eleicao-em-numeros/noticia/2018/08/23/rj-e-sp-sao-os-estados-com-a-maior-competicao-por-vaga-no-legislativo.ghtml

**DEPUTADOS FEDERAIS**. 2014 e 2018.  Necessitamos: 
 -  considerar por estado: podemos fazer isto só p o Rio, para começar. São duas as perguntas que queremos responder: os partidos costumam usar/preencher todas as vagas a q tem direito? Se não, a "culpa" é meio das botas? Ou seja quão distante fica um partido de preencher esse total quando olhamos quantos homens e quantas mulheres foram candidatos por aquele partido . Para tanto:

    a) quantidade de vagas oficiais a que o estado tem direito. No Brasil variam de 70  em São Paulo a um mínimo de 8 em vários estados;
    
    b) acrescentar a isso , para efeito de verificação do uso do máximo de vagas de candidaturas, 50% para cada partido que concorre sozinho. Por exemplo, se 8, vamos olhar o máximo de 12 vagas para  "partido isolado"; e 16 vagas nos casos de coligação( respectivamente 150 e 200% do número de cadeiras disponíveis no estado);
    
Vamos primeiro ver o total de vagas usadas por cada partido para candidaturas. Quantos partidos concorreram no estado? Quantos usaram todas as vagas - de candidatos? 
    - média e mediana ?
    - a seguir, qual a distância que o partido ficou de ocupar a o total de vagas? Média e mediana?  E será q essa distância geralmente é maior ou menor do que 30%? Por que a pergunta? Por que 30% é a cota obrigatória de ser preenchida por mulheres.
    
Queremos chegar só seguinte: no Brasil, ou no Rio, para começar, em geral x% dos partidos preenchem o total de vagas, ou , o contrário, o padrão tem sido não preencher esse total, ficando muito ou pouco distantes.

Hipótese simplificada: se for padrão que ou expressivo que a maior parte deixa uma sobra maior q 30%, então há vagas demais em oferta, ou, "a culpa" não é das mulheres. Não das botas! Das mulheres!

In [22]:
df_vagas = pd.read_csv('relacao_vagas_dep_federal_2018.csv', usecols=['UF','Vagas'])

df_vagas.head()

,UF,Vagas
0,AC,8
1,AL,9
2,AM,8
3,AP,8
4,BA,39


In [21]:
df_vagas[df_vagas['UF'] == 'RJ']

,UF,Vagas
18,RJ,46


In [82]:
df.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,08/04/2021,11:38:59,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000003276,13819,JAQUELINE MARQUES TEIXEIRA,JAQUELINE MARQUES,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,13,PT,PARTIDO DOS TRABALHADORES,190000000010,PARTIDO ISOLADO,PT,-4,NÃO DIVULGÁVEL,Não divulgável,-3,NÃO DIVULGÁVEL,NaN,nan,-4,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,NÃO DIVULGÁVEL,-4,5,SUPLENTE,Não divulgável,Não divulgável,1047852014,37404120146190000,2,DEFERIDO,2,DEFERIDO,SIM
1,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,AC,AC,ACRE,7,DEPUTADO ESTADUAL,10000000131,65565,LUZIELE ALVES DIAS,LUZIELE,#NULO#,19682140200,PCDOBACRI@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,65,PC do B,PARTIDO COMUNISTA DO BRASIL,10000000024,FRENTE POPULAR DO ACRE VI,PC do B / PTB,1,BRASILEIRA NATA,AC,-3,RIO BRANCO,06/05/1970,44.00,1743942453,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),3,PARDA,266,PROFESSOR DE ENSINO MÉDIO,600000,5,SUPLENTE,N,S,52882014,1981620146010000,2,DEFERIDO,2,DEFERIDO,SIM
2,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RS,RS,RIO GRANDE DO SUL,6,DEPUTADO FEDERAL,210000000667,4567,LEUDO IRAJA SANTOS COSTA,LEUDO COSTA,#NULO#,20529457091,LEUDOCOSTA@MEIOAMBIENTELEGAL.COM.BR,12,APTO,2,DEFERIDO,COLIGAÇÃO,45,PSDB,PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA,210000000020,UNIDOS PELA ESPERANÇA,PP / PRB / SD / PSDB,1,BRASILEIRA NATA,RS,-3,SANTIAGO,19/10/1955,59.00,37851190450,2,MASCULINO,8,SUPERIOR COMPLETO,9,DIVORCIADO(A),1,BRANCA,131,ADVOGADO,4500000,5,SUPLENTE,N,S,315182014,5147120146210000,2,DEFERIDO,2,DEFERIDO,SIM
3,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,GO,GO,GOIÁS,7,DEPUTADO ESTADUAL,90000000500,50111,ARNALDO LUIZ DA SILVA,ARNALDINHO,#NULO#,49062948120,SOLANOCASTRO50@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,50,PSOL,PARTIDO SOCIALISMO E LIBERDADE,90000000021,PARTIDO ISOLADO,PSOL,1,BRASILEIRA NATA,PE,-3,SAO BEBEDITO DO SUL,31/07/1964,50.00,39479221066,2,MASCULINO,3,ENSINO FUNDAMENTAL INCOMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,200000,4,NÃO ELEITO,N,S,303612014,7627020146090000,2,DEFERIDO,2,DEFERIDO,SIM
4,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,7,DEPUTADO ESTADUAL,190000000746,70038,JOSÉ JAILSON CRUZ,CAMARÃO,#NULO#,39653501704,JAILSONJCRUZ@IG.COM.BR,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,70,PT do B,PARTIDO TRABALHISTA DO BRASIL,190000000015,PARTIDO ISOLADO,PT do B,1,BRASILEIRA NATA,AL,-3,PENEDO,28/08/1952,62.00,49091390353,2,MASCULINO,7,SUPERIOR INCOMPLETO,3,CASADO(A),1,BRANCA,999,OUTROS,1000000,5,SUPLENTE,N,N,759262014,27392120146190000,2,DEFERIDO,2,DEFERIDO,SIM


In [83]:
df[
    (df['DS_CARGO'] == 'DEPUTADO FEDERAL') &
    (df['SG_UF'] == 'RJ')
].head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
33,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,6,DEPUTADO FEDERAL,190000000394,7773,ANTONIO CARLOS DA SILVA PASSOS,PASTOR DIDA,#NULO#,80421466715,#NULO#,12,APTO,2,DEFERIDO,COLIGAÇÃO,77,SD,SOLIDARIEDADE,190000000007,RIO SOLIDÁRIO,SD / PSL,1,BRASILEIRA NATA,RJ,-3,MESQUITA,02/02/1965,49.00,88924780353,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,3,CASADO(A),3,PARDA,999,OUTROS,5000000,5,SUPLENTE,N,S,755282014,6615420146190000,2,DEFERIDO,4,INDEFERIDO COM RECURSO,SIM
101,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,6,DEPUTADO FEDERAL,190000001878,2200,JOAO FERREIRA NETO,DR JOÃO,#NULO#,26144735704,NETOJ2004@GLOBO.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,22,PR,PARTIDO DA REPÚBLICA,190000000026,PR E PROS,PR / PROS,1,BRASILEIRA NATA,RJ,-3,RIO DE JANEIRO,05/08/1950,64.00,52862310337,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),3,PARDA,111,MÉDICO,3000000,2,ELEITO POR QP,N,S,769422014,21425220146190000,2,DEFERIDO,2,DEFERIDO,SIM
160,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,6,DEPUTADO FEDERAL,190000002167,2705,ROGÉRIO VERÍSSIMO RODRIGUES,ROGÉRIO VERISSIMO,#NULO#,4471670735,ROGERIOVERISSIMO01@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,27,PSDC,PARTIDO SOCIAL DEMOCRATA CRISTÃO,190000000029,POR UM RIO MELHOR,PSDC / PMN / PTC,1,BRASILEIRA NATA,RJ,-3,RIO DE JANEIRO,17/12/1974,40.00,83998600388,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),3,PARDA,999,OUTROS,2000000,5,SUPLENTE,N,N,773532014,13587520146190000,4,INDEFERIDO COM RECURSO,4,INDEFERIDO COM RECURSO,SIM
163,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,6,DEPUTADO FEDERAL,190000001384,1045,EDMILSON DA SILVA PAULINO,EDMILSON PALMIER,#NULO#,77349539720,EDMILSONPALMIER.AGEL@HOTMAIL.COM,3,INAPTO,14,INDEFERIDO,PARTIDO ISOLADO,10,PRB,PARTIDO REPUBLICANO BRASILEIRO,190000000019,PARTIDO ISOLADO,PRB,1,BRASILEIRA NATA,RJ,-3,RIO DE JANEIRO,06/04/1964,50.00,55667780337,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),1,BRANCA,169,COMERCIANTE,1800000,4,NÃO ELEITO,N,N,764812014,16298420146190000,4,INDEFERIDO COM RECURSO,4,INDEFERIDO COM RECURSO,SIM
233,08/04/2021,11:38:25,2014,2,ELEIÇÃO ORDINÁRIA,1,143,Eleições Gerais 2014,05/10/2014,FEDERAL,RJ,RJ,RIO DE JANEIRO,6,DEPUTADO FEDERAL,190000000380,1788,VERA LUCIA FIGUEIREDO DOS SANTOS,VERA FIGUEREDO,#NULO#,56674376787,VERAFIGUEIREDO@UOL.COM.BR,3,INAPTO,6,RENÚNCIA,COLIGAÇÃO,17,PSL,PARTIDO SOCIAL LIBERAL,190000000007,RIO SOLIDÁRIO,SD / PSL,1,BRASILEIRA NATA,RJ,-3,RIO DE JANEIRO,31/08/1943,71.00,19946490302,4,FEMININO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,266,PROFESSOR DE ENSINO MÉDIO,-1,-1,#NULO#,N,N,755542014,6875220146190000,-1,#NULO#,-1,#NULO#,NÃO


46 vagas pro rio

In [85]:
94/46

2.0434782608695654

In [88]:
df[
    (df['DS_CARGO'] == 'DEPUTADO FEDERAL') &
    (df['SG_UF'] == 'RJ')
].groupby(['TP_AGREMIACAO','NM_COLIGACAO'])['DS_GENERO'].value_counts()

TP_AGREMIACAO    NM_COLIGACAO                          DS_GENERO     
COLIGAÇÃO        FRENTE POPULAR I                      MASCULINO          59
                                                       FEMININO           26
                 FRENTE TRABALHISTA SOLIDARIA          MASCULINO          65
                                                       FEMININO           28
                 PACTO PELO RIO DE JANEIRO E O BRASIL  MASCULINO          66
                                                       FEMININO           28
                 PMDB - PP - PSC - PSD - PTB           MASCULINO          64
                                                       FEMININO           27
                 POR UM RIO MELHOR                     MASCULINO          65
                                                       FEMININO           29
                 PR E PROS                             MASCULINO          57
                                                       FEMININO           27
      

# Vereadores

## 2016 (com e sem coligação)

In [3]:
path = 'consulta_cand_2016/consulta_cand_2016_BRASIL.csv'
df_2016 = pd.read_csv(path, sep=';', encoding='latin1', low_memory=False, decimal=',')

df_2016.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,18/02/2021,12:07:20,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PR,76635,LOANDA,13,VEREADOR,160000024065,15567,EDUARDO FRANCISCO DA SILVA,EDUARDO CHAVEIRO,#NULO#,43796141900,PARANACONTABIL@HOTMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,15,PMDB,PARTIDO DO MOVIMENTO DEMOCRÁTICO BRASILEIRO,160000001804,PMDB-PTB-PT,PMDB / PTB / PT,1,BRASILEIRA NATA,PR,-3,NOVA LONDRINA,13/10/1957,59.00,22742330663,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),1,BRANCA,217,OFFICE-BOY E CONTÍNUO,10803.91,5,SUPLENTE,N,S,1148802016,2523020166160085,2,DEFERIDO,2,DEFERIDO,SIM
1,18/02/2021,12:07:20,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,CE,13897,FORTALEZA,13,VEREADOR,60000002469,40563,FRANCISCO DE ASSIS FERREIRA DO NASCIMENTO,IRMÃO FRANCISCO,#NULO#,08811300363,CONSUELODFERREIRA@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,40,PSB,PARTIDO SOCIALISTA BRASILEIRO,60000000188,VOCÊ PODE ACREDITAR,REDE / PSB,1,BRASILEIRA NATA,CE,-3,QUIXADÁ,07/12/1949,67.00,3737090787,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),1,BRANCA,999,OUTROS,460018.94,4,NÃO ELEITO,N,N,532302016,2027820166060118,2,DEFERIDO,2,DEFERIDO,SIM
2,18/02/2021,12:07:20,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,SP,71218,SÃO VICENTE,13,VEREADOR,250000054402,36111,ALESSANDRO ZACCHA DE ARAUJO,ALESSANDRO GIGANTE,#NULO#,16961025889,ALESSANDRO_GIGANTE@OUTLOOK.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,36,PTC,PARTIDO TRABALHISTA CRISTÃO,250000003318,PARTIDO ISOLADO,PTC,1,BRASILEIRA NATA,SP,-3,SANTOS,11/03/1971,45.00,173157390141,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,1,SOLTEIRO(A),1,BRANCA,254,VIGILANTE,243964.93,4,NÃO ELEITO,N,S,2127532016,2560620166260177,2,DEFERIDO,2,DEFERIDO,SIM
3,18/02/2021,12:07:20,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PB,19283,SOSSÊGO,13,VEREADOR,150000008781,20999,KATYUSKA DE SOUZA MARIA CARLOS,KAKÁ,#NULO#,08592763410,KATYUSKASOUZA@HOTMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,20,PSC,PARTIDO SOCIAL CRISTÃO,150000000680,A FORÇA DA MUDANÇA,PSDB / PTB / PSC / SD,1,BRASILEIRA NATA,PB,-3,CUBATI,09/07/1992,24.00,40468001236,4,FEMININO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),3,PARDA,999,OUTROS,10803.91,5,SUPLENTE,N,S,533062016,2273320166150024,2,DEFERIDO,2,DEFERIDO,SIM
4,18/02/2021,12:07:20,2016,2,ELEIÇÃO ORDINÁRIA,1,220,Eleições Municipais 2016,02/10/2016,MUNICIPAL,PE,25119,PAUDALHO,13,VEREADOR,170000018018,23123,MARINALDO ANSELMO DA SILVA,TOURO DO POVO,#NULO#,59232242400,FRENTEPOPULARDOPAUDALHO@GMAIL.COM,12,APTO,2,DEFERIDO,COLIGAÇÃO,23,PPS,PARTIDO POPULAR SOCIALISTA,170000001139,PARA CONTINUAR AVANÇANDO,PPS / DEM / PSDC / PR,1,BRASILEIRA NATA,PE,-3,PAUDALHO,20/09/1968,48.00,59432750868,2,MASCULINO,4,ENSINO FUNDAMENTAL COMPLETO,1,SOLTEIRO(A),3,PARDA,999,OUTROS,15664.78,5,SUPLENTE,N,N,670712016,1283020166170017,2,DEFERIDO,2,DEFERIDO,SIM


In [41]:
df_2016.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
       'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO', 'NM_EMAIL',
       'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'CD_NACIONALIDADE',
       'DS_NACIONALIDADE', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO',
       'NM_MUNICIPIO_NASCIMENTO', 'DT_NASCIMENTO', 'NR_IDADE_DATA_POSSE',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
       'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO',
       '

### Candidatos a vereador em 2016

Abaixo estão as coligações presentes em cada município e a quantidade de candidaturas por gênero em cada coligação, além da quantidade de candidaturas por gênero representadas por partidos isolados:

MUNICIPIO   NOME_COLIGACAO    NO HOMENS    NO MULHERES    TOTAL    PERCENT QUE ISSO REPRESENTA DAS VAGAS TOTAIS

In [12]:
df_vereador_coligacoes_genero_2016  = df_2016[
    (df_2016['DS_CARGO'] == 'VEREADOR') &
    (df_2016['TP_AGREMIACAO'] == 'COLIGAÇÃO')
].groupby(['SG_UF', 'NM_UE'])['NM_COLIGACAO'].value_counts()

df_vereador_coligacoes_genero_2016.head(10)

NameError: name 'df_2016' is not defined

In [ ]:
df_vereador_coligacoes_genero_2016["DS_"]

In [26]:
df_vereador_coligacoes_genero_2016  = df_2016[
    (df_2016['DS_CARGO'] == 'VEREADOR') &
    (df_2016['TP_AGREMIACAO'] == 'PARTIDO ISOLADO')
].groupby(['SG_UF','NM_UE','NM_PARTIDO'])['DS_GENERO'].value_counts()

df_vereador_coligacoes_genero_2016.head()

SG_UF  NM_UE         NM_PARTIDO                               DS_GENERO
AC     ACRELÂNDIA    PARTIDO DA SOCIAL DEMOCRACIA BRASILEIRA  MASCULINO    7
                                                              FEMININO     3
                     PARTIDO DOS TRABALHADORES                MASCULINO    5
                                                              FEMININO     3
       ASSIS BRASIL  PARTIDO COMUNISTA DO BRASIL              MASCULINO    8
Name: DS_GENERO, dtype: int64

## 2020 (sem coligação)

"Pela primeira vez, partidos políticos não poderão se coligar para concorrerem ao cargo de vereador, podendo apresentar pedidos somente de forma isolada".  A nova regra foi fixada pela Emenda Constitucional nº 97/2017.

https://www.tse.jus.br/imprensa/noticias-tse/2020/Novembro/eleicoes-2020-58-208-vagas-de-vereadores-estarao-em-disputa-neste-domingo-15

### Vagas disponíveis para vereador por município

In [18]:
df_vagas_2020 = pd.read_csv('relacao_vaga_2020.csv', sep=';', encoding='latin1', low_memory=False, usecols=['UF', 'Município', 'Cargo', 'Quantitativo', 'Vaga'])

df_vereador_2020 = df_vagas_2020[df_vagas_2020['Cargo'] == 'Vereador']

df_vereador_2020 = df_vereador_2020.rename(columns={'Município':'NM_UE'});

In [20]:
df_vereador_2020.head(10)

,UF,NM_UE,Cargo,Quantitativo,Vaga
11136,AC,ACRELÂNDIA,Vereador,79,9
11137,AC,ASSIS BRASIL,Vereador,85,9
11138,AC,BRASILÉIA,Vereador,136,11
11139,AC,BUJARI,Vereador,130,9
11140,AC,CAPIXABA,Vereador,104,9
11141,AC,CRUZEIRO DO SUL,Vereador,209,14
11142,AC,EPITACIOLÂNDIA,Vereador,127,9
11143,AC,FEIJÓ,Vereador,141,13
11144,AC,JORDÃO,Vereador,74,9
11145,AC,MÂNCIO LIMA,Vereador,125,11


In [3]:
path = 'consulta_cand_2020/consulta_cand_2020_BRASIL.csv'
df_2020 = pd.read_csv(path, sep=';', encoding='latin1', low_memory=False, decimal=',')

df_2020.head()

,DT_GERACAO,HH_GERACAO,ANO_ELEICAO,CD_TIPO_ELEICAO,NM_TIPO_ELEICAO,NR_TURNO,CD_ELEICAO,DS_ELEICAO,DT_ELEICAO,TP_ABRANGENCIA,SG_UF,SG_UE,NM_UE,CD_CARGO,DS_CARGO,SQ_CANDIDATO,NR_CANDIDATO,NM_CANDIDATO,NM_URNA_CANDIDATO,NM_SOCIAL_CANDIDATO,NR_CPF_CANDIDATO,NM_EMAIL,CD_SITUACAO_CANDIDATURA,DS_SITUACAO_CANDIDATURA,CD_DETALHE_SITUACAO_CAND,DS_DETALHE_SITUACAO_CAND,TP_AGREMIACAO,NR_PARTIDO,SG_PARTIDO,NM_PARTIDO,SQ_COLIGACAO,NM_COLIGACAO,DS_COMPOSICAO_COLIGACAO,CD_NACIONALIDADE,DS_NACIONALIDADE,SG_UF_NASCIMENTO,CD_MUNICIPIO_NASCIMENTO,NM_MUNICIPIO_NASCIMENTO,DT_NASCIMENTO,NR_IDADE_DATA_POSSE,NR_TITULO_ELEITORAL_CANDIDATO,CD_GENERO,DS_GENERO,CD_GRAU_INSTRUCAO,DS_GRAU_INSTRUCAO,CD_ESTADO_CIVIL,DS_ESTADO_CIVIL,CD_COR_RACA,DS_COR_RACA,CD_OCUPACAO,DS_OCUPACAO,VR_DESPESA_MAX_CAMPANHA,CD_SIT_TOT_TURNO,DS_SIT_TOT_TURNO,ST_REELEICAO,ST_DECLARAR_BENS,NR_PROTOCOLO_CANDIDATURA,NR_PROCESSO,CD_SITUACAO_CANDIDATO_PLEITO,DS_SITUACAO_CANDIDATO_PLEITO,CD_SITUACAO_CANDIDATO_URNA,DS_SITUACAO_CANDIDATO_URNA,ST_CANDIDATO_INSERIDO_URNA
0,29/05/2021,12:19:11,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,65099,ILHABELA,13,VEREADOR,250001078745,70565,JOSE PEREIRA BRANDÃO,ZÉ BRANDÃO,#NULO#,5400594808,ILHAVIVA2020@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,70,AVANTE,AVANTE,250000130531,PARTIDO ISOLADO,AVANTE,1,BRASILEIRA NATA,MG,-3,GRÃO MOGOL,08/03/1958,62,47636980108,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,9,DIVORCIADO(A),3,PARDA,709,TRABALHADOR DE CONSTRUÇÃO CIVIL,12307.75,5,SUPLENTE,N,S,-1,6005873520206260132,2,DEFERIDO,2,DEFERIDO,SIM
1,29/05/2021,12:19:11,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,BA,37567,NOVA FÁTIMA,13,VEREADOR,50000941067,11135,ERIVANIA ARAUJO DA SILVA,KAL ARAUJO,#NULO#,3279617596,UNIDOSPORNOVAFATIMA@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,11,PP,PROGRESSISTAS,50000108688,PARTIDO ISOLADO,PP,1,BRASILEIRA NATA,BA,-3,RIACHÃO DO JACUÍPE,01/04/1986,34,116352710507,4,FEMININO,7,SUPERIOR INCOMPLETO,9,DIVORCIADO(A),3,PARDA,254,VIGILANTE,12307.75,5,SUPLENTE,N,S,-1,6001862320206050196,2,DEFERIDO,2,DEFERIDO,SIM
2,29/05/2021,12:19:11,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,PB,19763,ALCANTIL,13,VEREADOR,150000859227,12112,ARLETE BARBOSA COSTA,ARLETE,#NULO#,26253330497,JOAO.RODRIGUES.SEGUNDO@OUTLOOK.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,12,PDT,PARTIDO DEMOCRÁTICO TRABALHISTA,150000093695,PARTIDO ISOLADO,PDT,1,BRASILEIRA NATA,PB,-3,BOQUEIRÃO,12/02/1959,61,11174261295,4,FEMININO,6,ENSINO MÉDIO COMPLETO,3,CASADO(A),3,PARDA,999,OUTROS,12307.75,5,SUPLENTE,N,S,-1,6001080220206150018,2,DEFERIDO,2,DEFERIDO,SIM
3,29/05/2021,12:19:11,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,70971,SÃO JOSÉ DO RIO PRETO,13,VEREADOR,250001178195,11123,DAVID MARCELINO CARDOZO,DAVID CARDOZO,#NULO#,6451512893,DAVI.CARDOZO@HOTMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,11,PP,PROGRESSISTAS,250000145024,PARTIDO ISOLADO,PP,1,BRASILEIRA NATA,SP,-3,SÃO JOSÉ DO RIO PRETO,07/05/1965,55,58495610141,2,MASCULINO,8,SUPERIOR COMPLETO,3,CASADO(A),1,BRANCA,133,SOCIÓLOGO,113318.61,5,SUPLENTE,N,S,-1,6004308320206260125,2,DEFERIDO,2,DEFERIDO,SIM
4,29/05/2021,12:19:11,2020,2,ELEIÇÃO ORDINÁRIA,1,426,Eleições Municipais 2020,15/11/2020,MUNICIPAL,SP,71072,SÃO PAULO,13,VEREADOR,250001159238,90333,MANOEL MILANEZ DA SILVA FILHO,MILANEZ,#NULO#,84883987434,MILANEZEFMANO@GMAIL.COM,12,APTO,2,DEFERIDO,PARTIDO ISOLADO,90,PROS,PARTIDO REPUBLICANO DA ORDEM SOCIAL,250000140997,PARTIDO ISOLADO,PROS,1,BRASILEIRA NATA,SP,-3,SÃO PAULO,04/11/1973,47,40986080809,2,MASCULINO,6,ENSINO MÉDIO COMPLETO,1,SOLTEIRO(A),1,BRANCA,257,EMPRESÁRIO,3675197.12,4,NÃO ELEITO,N,N,-1,6019843720206260001,2,DEFERIDO,2,DEFERIDO,SIM


In [23]:
df_vereador_sem_coligacoes_genero_feminino_2020 = df_2020[
    (df_2020['DS_CARGO'] == 'VEREADOR') &
    (df_2020['DS_GENERO'] == 'FEMININO')
].groupby(['SG_UF','NM_UE', 'SG_PARTIDO'], as_index = False)['DS_GENERO'].count()

In [24]:
df_vereador_sem_coligacoes_genero_feminino_2020 = df_vereador_sem_coligacoes_genero_feminino_2020.rename(columns={'DS_GENERO':'GENERO_FEMININO'});

In [29]:
df_vereador_sem_coligacoes_genero_feminino_2020.head()

,SG_UF,NM_UE,SG_PARTIDO,GENERO_FEMININO,GENERO_MASCULINO
0,AC,ACRELÂNDIA,DEM,1,1
1,AC,ACRELÂNDIA,MDB,4,8
2,AC,ACRELÂNDIA,PDT,1,2
3,AC,ACRELÂNDIA,PL,3,6
4,AC,ACRELÂNDIA,PP,5,9


In [26]:
df_vereador_sem_coligacoes_genero_masculino_2020 = df_2020[
    (df_2020['DS_CARGO'] == 'VEREADOR') &
    (df_2020['DS_GENERO'] == 'MASCULINO')
].groupby(['SG_UF','NM_UE', 'SG_PARTIDO'], as_index = False)['DS_GENERO'].count()

In [27]:
df_vereador_sem_coligacoes_genero_masculino_2020.head()

,SG_UF,NM_UE,SG_PARTIDO,DS_GENERO
0,AC,ACRELÂNDIA,DEM,1
1,AC,ACRELÂNDIA,MDB,8
2,AC,ACRELÂNDIA,PDT,2
3,AC,ACRELÂNDIA,PL,6
4,AC,ACRELÂNDIA,PP,9


In [28]:
df_vereador_sem_coligacoes_genero_feminino_2020['GENERO_MASCULINO'] = df_vereador_sem_coligacoes_genero_masculino_2020['DS_GENERO']

In [34]:
df_vereador_sem_coligacoes_por_genero_2020 = pd.merge(df_vereador_sem_coligacoes_genero_feminino_2020, df_vereador_2020[['NM_UE', 'Quantitativo', 'Vaga']] , on = 'NM_UE')

In [35]:
df_vereador_sem_coligacoes_por_genero_2020.head()

,SG_UF,NM_UE,SG_PARTIDO,GENERO_FEMININO,GENERO_MASCULINO,Quantitativo,Vaga
0,AC,ACRELÂNDIA,DEM,1,1,79,9
1,AC,ACRELÂNDIA,MDB,4,8,79,9
2,AC,ACRELÂNDIA,PDT,1,2,79,9
3,AC,ACRELÂNDIA,PL,3,6,79,9
4,AC,ACRELÂNDIA,PP,5,9,79,9


In [22]:
df_vereador_sem_coligacoes_por_genero_2020.head()

,SG_UF,NM_UE,SG_PARTIDO,GENERO_MASCULINO,UF,Cargo,Quantitativo,Vaga
0,AC,ACRELÂNDIA,DEM,1,AC,Vereador,79,9
1,AC,ACRELÂNDIA,MDB,8,AC,Vereador,79,9
2,AC,ACRELÂNDIA,PDT,2,AC,Vereador,79,9
3,AC,ACRELÂNDIA,PL,6,AC,Vereador,79,9
4,AC,ACRELÂNDIA,PP,9,AC,Vereador,79,9


MUNICIPIO NOME_PARTIDO  NOHOMENS NOMULHERES TOTAL PERCENTDASVAGASDISPONIVEIS NUMERODEVAGASDISPONIVEIS

In [16]:
df_2020.columns

Index(['DT_GERACAO', 'HH_GERACAO', 'ANO_ELEICAO', 'CD_TIPO_ELEICAO',
       'NM_TIPO_ELEICAO', 'NR_TURNO', 'CD_ELEICAO', 'DS_ELEICAO', 'DT_ELEICAO',
       'TP_ABRANGENCIA', 'SG_UF', 'SG_UE', 'NM_UE', 'CD_CARGO', 'DS_CARGO',
       'SQ_CANDIDATO', 'NR_CANDIDATO', 'NM_CANDIDATO', 'NM_URNA_CANDIDATO',
       'NM_SOCIAL_CANDIDATO', 'NR_CPF_CANDIDATO', 'NM_EMAIL',
       'CD_SITUACAO_CANDIDATURA', 'DS_SITUACAO_CANDIDATURA',
       'CD_DETALHE_SITUACAO_CAND', 'DS_DETALHE_SITUACAO_CAND', 'TP_AGREMIACAO',
       'NR_PARTIDO', 'SG_PARTIDO', 'NM_PARTIDO', 'SQ_COLIGACAO',
       'NM_COLIGACAO', 'DS_COMPOSICAO_COLIGACAO', 'CD_NACIONALIDADE',
       'DS_NACIONALIDADE', 'SG_UF_NASCIMENTO', 'CD_MUNICIPIO_NASCIMENTO',
       'NM_MUNICIPIO_NASCIMENTO', 'DT_NASCIMENTO', 'NR_IDADE_DATA_POSSE',
       'NR_TITULO_ELEITORAL_CANDIDATO', 'CD_GENERO', 'DS_GENERO',
       'CD_GRAU_INSTRUCAO', 'DS_GRAU_INSTRUCAO', 'CD_ESTADO_CIVIL',
       'DS_ESTADO_CIVIL', 'CD_COR_RACA', 'DS_COR_RACA', 'CD_OCUPACAO',
       '